## Movie_Review_Sentiment_Classification
### (영화리뷰 감성분석)

***
#### 1. 데이터 준비와 확인

In [2]:
import random
import numpy as np
import pandas as pd
import torch
from torchtext import data
from torchtext import datasets
from tqdm import tqdm

from eunjeon import Mecab
mecab = Mecab()

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

문장의 길이가 필터 사이즈보다 작으면 에러가 나므로 다음과 같이 토크나이저 수정

In [3]:
FILTER_SIZES = [3,4,5]
def tokenizer(text):
    token = [t for t in mecab.morphs(text)]
    if len(token) < max(FILTER_SIZES):
        for i in range(0, max(FILTER_SIZES) - len(token)):
            token.append('<PAD>')
    return token

In [4]:
TEXT = data.Field(tokenize = tokenizer, batch_first = True)
LABEL = data.LabelField(dtype = torch.float)

In [5]:
fields = {'document': ('text',TEXT), 'label': ('label',LABEL)}
# dictionary 형식은 {csv컬럼명 : (데이터 컬럼명, Field이름)}

In [6]:
df1 = pd.read_csv("0112.csv", encoding="utf-8")
df2 = pd.read_csv("0113.csv", encoding="utf-8")
df3 = pd.read_csv("0114.csv", encoding="utf-8")
df4 = pd.read_csv("0115.csv", encoding="utf-8")
df5 = pd.read_csv("0116.csv", encoding="utf-8")
df6 = pd.read_csv("0117.csv", encoding="utf-8")
df7 = pd.read_csv("0118.csv", encoding="utf-8")
df8 = pd.read_csv("0119.csv", encoding="utf-8")
df9 = pd.read_csv("0120.csv", encoding="utf-8")
df10 = pd.read_csv("0121.csv", encoding="utf-8")
df11 = pd.read_csv("0122.csv", encoding="utf-8")
df12 = pd.read_csv("0123.csv", encoding="utf-8")
df13 = pd.read_csv("0124.csv", encoding="utf-8")
df14 = pd.read_csv("0125.csv", encoding="utf-8")
df15 = pd.read_csv("0126.csv", encoding="utf-8")
df16 = pd.read_csv("0127.csv", encoding="utf-8")
df17 = pd.read_csv("0128.csv", encoding="utf-8")
df18 = pd.read_csv("0129.csv", encoding="utf-8")
df19 = pd.read_csv("0130.csv", encoding="utf-8")
df20 = pd.read_csv("0131.csv", encoding="utf-8")
df21 = pd.read_csv("0201.csv", encoding="utf-8")
df22 = pd.read_csv("0202.csv", encoding="utf-8")
df23 = pd.read_csv("0203.csv", encoding="utf-8")
df24 = pd.read_csv("0204.csv", encoding="utf-8")
df25 = pd.read_csv("0205.csv", encoding="utf-8")
df26 = pd.read_csv("0206.csv", encoding="utf-8")
df27 = pd.read_csv("0207.csv", encoding="utf-8")
df28 = pd.read_csv("0208.csv", encoding="utf-8")
df29 = pd.read_csv("0209.csv", encoding="utf-8")
df30 = pd.read_csv("0210.csv", encoding="utf-8")
df31 = pd.read_csv("0211.csv", encoding="utf-8")
df32 = pd.read_csv("0212.csv", encoding="utf-8")
df33 = pd.read_csv("0213.csv", encoding="utf-8")

In [7]:
df = pd.concat([df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17,df18,df19,df20,
                df21,df22,df23,df24,df25,df26,df27,df28,df29,df30,df31,df32,df33], axis=0, ignore_index = True)
df = df.rename(columns = {'sentence':'document'})

def rating_to_label(score):
    if score <= 2:
        return 1
    if 3 <= score <= 4:
        return 2
    if 5 <= score <= 6:
        return 3
    if 7 <= score <= 8:
        return 4
    if score >= 9:
        return 5
    
df['label'] = df['score'].apply(lambda x: rating_to_label(x))
df = df.drop(columns = 'score')
df['document'] = df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
df['document'] = df['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
df['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
df = df.dropna(axis = 0)
df.set_index('document',inplace=True)
df = df.astype('int')
df.reset_index(inplace=True)

C:\Users\user\AppData\Local\Temp\ipykernel_27984\3171784776.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df['document'] = df['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
C:\Users\user\AppData\Local\Temp\ipykernel_27984\3171784776.py:20: FutureWarning: The default value of regex will change from True to False in a future version.
  df['document'] = df['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경


In [8]:
df.label.value_counts()

5    43890
4    12477
1    10026
3     6496
2     3591
Name: label, dtype: int64

In [9]:
df = pd.concat((df[df['label']==1].head(len(df[df['label']==2])),df[df['label']==2],df[df['label']==3].head(len(df[df['label']==2])),
                df[df['label']==4].head(len(df[df['label']==2])),df[df['label']==5].head(len(df[df['label']==2]))), axis=0, ignore_index = True)
df.head()

,document,label
0,이걸 영화라고,1
1,절대 보지 마세요완전 지루하고 노잼입니다평점이 도저히 이해가 안갑니다돈이 너무 아깝...,1
2,그렇게 하고싶냐,1
3,전개성은 비행기 이륙하는 순간 떨군 것 같아요 좋은 배우분들로 이런 영화를 찍은게 ...,1
4,그냥 일본만화 ㅋㅋ,1


In [10]:
df.label.value_counts()

1    3591
2    3591
3    3591
4    3591
5    3591
Name: label, dtype: int64

In [11]:
stopwords = ['은','는','이','가','의','들','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

def preprocess(text):
    word = [t for t in text if t not in stopwords]
    return word

In [12]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, random_state=0)

train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [13]:
train_data.to_csv("train_data.csv", index=False)
test_data.to_csv("test_data.csv", index=False)

In [14]:
for sentence in tqdm(train_data['document']):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    train_data.append(stopwords_removed_sentence)

  0%|                                                                                        | 0/14364 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_27984\3166225607.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  train_data.append(stopwords_removed_sentence)
100%|███████████████████████████████████████████████████████████████████████████| 14364/14364 [00:31<00:00, 450.75it/s]


In [15]:
for sentence in tqdm(test_data['document']):
    tokenized_sentence = mecab.morphs(sentence) # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    test_data.append(stopwords_removed_sentence)

  0%|                                                                                         | 0/3591 [00:00<?, ?it/s]C:\Users\user\AppData\Local\Temp\ipykernel_27984\2256332727.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_data.append(stopwords_removed_sentence)
100%|█████████████████████████████████████████████████████████████████████████████| 3591/3591 [00:06<00:00, 571.21it/s]


In [16]:
import random
train_data, test_data = data.TabularDataset.splits(
                            path = '.',
                            train = 'train_data.csv',
                            test = 'test_data.csv',
                            format = 'csv',
                            fields = fields
)
train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [17]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                max_size = MAX_VOCAB_SIZE,
                vectors = 'fasttext.simple.300d',
                unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [18]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch = True,
    device = device)

In [19]:
import torch.nn as nn
import torch.nn.functional as F

In [20]:
def print_shape(name, data):
    print(f'{name} has shape {data.shape}')

In [21]:
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, dropout, pad_idx):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        self.convs = nn.ModuleList([nn.Conv2d(in_channels=1,
                                             out_channels=n_filters,
                                             kernel_size=(fs, embedding_dim))
                                   for fs in filter_sizes])
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        #print_shape('text', text)
        # text = [batch_size, sent_len]
        
        embedded = self.embedding(text)
        #print_shape('embedded', embedded)
        # embedded = [batch_size, sent_len, emb_dim]
        
        embedded = embedded.unsqueeze(1)
        #print_shape('embedded', embedded)
        # embedded = [batch_size, 1, sent_len, emb_dim]
        
        #print_shape('self.convs[0](embedded)', self.convs[0](embedded))
        # self.convs[0](embedded) = [batch_size, n_filters, sent_len-filter_sizes[n]+1, 1 ]
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
        
        #print_shape('F.max_pool1d(conved[0], conved[0].shape[2])', F.max_pool1d(conved[0], conved[0].shape[2]))
        # F.max_pool1d(conved[0], conved[0].shape[2]) = [batch_size, n_filters, 1]
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        cat = self.dropout(torch.cat(pooled, dim=1))
        #print_shape('cat', cat)
        # cat = [batch_size, n_filters * len(filter_size)]
        
        res = self.fc(cat)
        #print_shape('res', res)
        # res = [batch_size, output_dim]
        
        return self.fc(cat)

In [22]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 300
N_FILTERS = 100
FILTER_SIZES = [3,4,5]
OUTPUT_DIM = 1
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [23]:
model = model.to(device)

In [24]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'모델의 파라미터 수는 {count_parameters(model):,} 개 입니다.')

모델의 파라미터 수는 4,548,001 개 입니다.


In [25]:
pretrained_weight = TEXT.vocab.vectors
print(pretrained_weight.shape, model.embedding.weight.data.shape)

torch.Size([13958, 300]) torch.Size([13958, 300])


In [26]:
model.embedding.weight.data.copy_(pretrained_weight)

tensor([[-0.1117, -0.4966,  0.1631,  ..., -1.4447,  0.8402, -0.8668],
        [ 0.1032, -1.6268,  0.5729,  ...,  0.3180, -0.1626, -0.0417],
        [-0.1020,  1.6282,  2.1635,  ..., -0.6009, -0.1467,  0.0285],
        ...,
        [ 0.2466,  0.7686, -0.8748,  ...,  1.2832, -0.6463,  0.3351],
        [ 1.9161, -0.7881, -0.4273,  ..., -0.2970,  1.7449,  0.4935],
        [ 0.8965, -0.0574,  0.0136,  ..., -1.6213, -0.2140,  0.5201]])

In [27]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

## 모델 훈련

In [28]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [29]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [30]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds==y).float()
    acc = correct.sum() / len(correct)
    return acc

In [31]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        optimizer.zero_grad()
        predictions = model(batch.text).squeeze(1) # output_dim = 1
        loss = criterion(predictions, batch.label)
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [34]:
from tqdm import tqdm

N_EPOCHS = 5
best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
        
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

 20%|████████████████▊                                                                   | 1/5 [00:15<01:03, 15.98s/it]

Epoch: 01 | Epoch Time: 0m 15s
	Train Loss: -547.130 | Train Acc: 20.08%
	 Val. Loss: -2128.576 |  Val. Acc: 20.34%


 40%|█████████████████████████████████▌                                                  | 2/5 [00:32<00:48, 16.25s/it]

Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: -8910.389 | Train Acc: 20.00%
	 Val. Loss: -20789.856 |  Val. Acc: 20.34%


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:48<00:32, 16.37s/it]

Epoch: 03 | Epoch Time: 0m 16s
	Train Loss: -47590.695 | Train Acc: 20.00%
	 Val. Loss: -85282.599 |  Val. Acc: 20.34%


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:03<00:15, 15.83s/it]

Epoch: 04 | Epoch Time: 0m 14s
	Train Loss: -148046.317 | Train Acc: 20.00%
	 Val. Loss: -229327.555 |  Val. Acc: 20.34%


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:19<00:00, 15.94s/it]

Epoch: 05 | Epoch Time: 0m 15s
	Train Loss: -346242.571 | Train Acc: 20.08%
	 Val. Loss: -490786.062 |  Val. Acc: 20.34%


In [35]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: -502401.632 | Test Acc: 19.19%


In [36]:
for epoch in tqdm(range(N_EPOCHS)):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut4-model.pt')
            
    print(f'Epoch: {epoch+6:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

 20%|████████████████▊                                                                   | 1/5 [00:17<01:09, 17.27s/it]

Epoch: 06 | Epoch Time: 0m 17s
	Train Loss: -671204.522 | Train Acc: 20.00%
	 Val. Loss: -890532.848 |  Val. Acc: 20.34%


 40%|█████████████████████████████████▌                                                  | 2/5 [00:35<00:54, 18.07s/it]

Epoch: 07 | Epoch Time: 0m 18s
	Train Loss: -1160749.428 | Train Acc: 20.00%
	 Val. Loss: -1462038.785 |  Val. Acc: 20.34%


 60%|██████████████████████████████████████████████████▍                                 | 3/5 [00:55<00:37, 18.56s/it]

Epoch: 08 | Epoch Time: 0m 19s
	Train Loss: -1805310.160 | Train Acc: 20.00%
	 Val. Loss: -2211162.774 |  Val. Acc: 20.34%


 80%|███████████████████████████████████████████████████████████████████▏                | 4/5 [01:14<00:18, 18.94s/it]

Epoch: 09 | Epoch Time: 0m 19s
	Train Loss: -2668784.336 | Train Acc: 20.00%
	 Val. Loss: -3162471.789 |  Val. Acc: 20.34%


100%|████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:34<00:00, 18.90s/it]

Epoch: 10 | Epoch Time: 0m 19s
	Train Loss: -3714372.308 | Train Acc: 20.08%
	 Val. Loss: -4322421.732 |  Val. Acc: 20.34%


In [37]:
model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: -4406501.320 | Test Acc: 19.19%
